In [1]:
import pandas as pd
import os

In [5]:
df = pd.read_csv('tests/set4_homooligomers.csv', sep='\t').head(10)

In [6]:
df = df.drop_duplicates(subset='full_sequence', keep='first')

In [7]:
df

,pdb,cc_cov,chains,cc_sequence,full_sequence,full_seq_corr,cc_seq_corr,parallel
0,3e7k,0.851852,4,TFERVEQMSIQIKEVGDRVNYIKRSLQSLDSQIGHLQDLSALTVDT,SRVTFERVEQMSIQIKEVGDRVNYIKRSLQSLDSQIGHLQDLSALT...,True,False,False
1,1use,0.550000,4,YSDLQRVKQELLEEVKKELQKV,SSDYSDLQRVKQELLEEVKKELQKVKEEIIEAFVQELRKR,False,False,True
2,6us8,0.826087,4,LVVAASIIGILHLILWILD,DPLVVAASIIGILHLILWILDRL,True,False,True
3,5kht,0.893617,2,DAIKKKMQMLKLDKENALDRAEQAEADNYHLENEVARLKKL,GMDAIKKKMQMLKLDKENALDRAEQAEADNYHLENEVARLKKLVGER,False,True,True
4,5c9n,0.671875,2,CSWEEAQLSSQLYRNKQLQDTLVQKEEELARLHEENNHLRQYL,LPDLCSWEEAQLSSQLYRNKQLQDTLVQKEEELARLHEENNHLRQY...,True,False,True
5,1c94,0.675676,2,LKKLRAVENELHYNKSLLEEVKDE,GGREGVLKKLRAVENELHYNKSLLEEVKDELQKMRQL,False,True,True
6,3a2a,0.878049,2,LLRLKQMNVQLAAKIQHLEFSCSEKEQEIERLNKLL,RQLLRLKQMNVQLAAKIQHLEFSCSEKEQEIERLNKLLRQH,False,False,True
7,7bji,0.829268,2,YKTENARLVKENNEMHLGLLKLREEKDRISRELKAYIRKLDHETSD...,NVDAILEPYKTENARLVKENNEMHLGLLKLREEKDRISRELKAYIR...,True,False,True
8,3bj4,0.891892,4,IGARLNRVEDKVTQLDQRLALITDMLHQLLSLH,SNTIGARLNRVEDKVTQLDQRLALITDMLHQLLSLHG,True,False,True
9,4w80,0.593750,4,EDQEKLKTELRKTSDALSKAQNDVMEMKMQSERLSK,ECVLEAENKKLVEDQEKLKTELRKTSDALSKAQNDVMEMKMQSERL...,True,True,False


In [12]:
for key, value in df.iterrows():
    print(key, value)
    os.makedirs(f'testcases/{key}', exist_ok=True)
    seq = value['full_sequence']
    pdb = value['pdb']
    
    with open(f'testcases/{key}/{key}_monomer.csv', 'w') as f:
        f.write('id,sequence\n')
        f.write(f'{pdb},{seq}\n')
        
    with open(f'testcases/{key}.sh', 'w') as f:
        f.write('#!/bin/bash\n')
        f.write(f'cd ./testcases/{key}\n')
        f.write('source /opt/miniconda3/bin/activate cf_1.5\n')
        f.write(f'colabfold_batch {key}_monomer.csv . --num-models 5 --model-type alphafold2_multimer_v3 --num-recycle 5 --save-single-representations --save-pair-representations\n')
        
    

0 pdb                                                           3e7k
cc_cov                                                    0.851852
chains                                                           4
cc_sequence         TFERVEQMSIQIKEVGDRVNYIKRSLQSLDSQIGHLQDLSALTVDT
full_sequence    SRVTFERVEQMSIQIKEVGDRVNYIKRSLQSLDSQIGHLQDLSALT...
full_seq_corr                                                 True
cc_seq_corr                                                  False
parallel                                                     False
Name: 0, dtype: object
1 pdb                                                  1use
cc_cov                                               0.55
chains                                                  4
cc_sequence                        YSDLQRVKQELLEEVKKELQKV
full_sequence    SSDYSDLQRVKQELLEEVKKELQKVKEEIIEAFVQELRKR
full_seq_corr                                       False
cc_seq_corr                                         False
parallel                       

In [10]:
len(df)

246

In [13]:
with open(f'testcases/batch.sh', 'w') as f:
    f.write("""#!/bin/bash
#SBATCH -p gpu
#SBATCH -n 4
#SBATCH --gres=gpu:1
#SBATCH --mem=16GB
#SBATCH -J wio
#SBATCH --array 0-10

bash $SLURM_ARRAY_TASK_ID.sh

""")